# PostgreSQL to Qdrant

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thierrypdamiba/qdrant-etl-cookbook/blob/main/notebooks/etl/postgres_to_qdrant.ipynb)

Extract records from PostgreSQL, embed text columns, and sync into Qdrant. This demo uses sqlite3 as a stand-in to keep it self-contained.

In [ ]:
!pip install -q qdrant-client sentence-transformers

In [ ]:
import sqlite3
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from sentence_transformers import SentenceTransformer

In [ ]:
client = QdrantClient(":memory:")
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
# Create sample database (replace with psycopg2 + PostgreSQL in production)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()
cursor.execute("CREATE TABLE products (id INTEGER PRIMARY KEY, title TEXT, description TEXT)")
cursor.executemany(
    "INSERT INTO products (id, title, description) VALUES (?, ?, ?)",
    [
        (1, "Vector Database", "A database optimized for storing and querying vector embeddings"),
        (2, "Search Engine", "Software for indexing and searching through large document collections"),
        (3, "ML Platform", "End-to-end platform for training and deploying machine learning models"),
        (4, "Data Pipeline", "ETL tool for extracting, transforming, and loading data between systems"),
        (5, "API Gateway", "Service that manages and routes API requests with authentication"),
    ],
)
conn.commit()
print("Sample database created")

In [ ]:
client.create_collection(
    collection_name="pg_data",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

In [ ]:
cursor.execute("SELECT id, title, description FROM products")

points = []
for row in cursor.fetchall():
    record_id, title, description = row
    text = f"{title} {description}"
    embedding = model.encode(text).tolist()
    points.append(
        PointStruct(
            id=record_id,
            vector=embedding,
            payload={"title": title, "description": description},
        )
    )

    if len(points) >= 100:
        client.upsert(collection_name="pg_data", points=points)
        points = []

if points:
    client.upsert(collection_name="pg_data", points=points)

conn.close()
print(f"Loaded {cursor.lastrowid} records from database")

In [ ]:
# Search
query_vector = model.encode("tool for processing data").tolist()
results = client.search(
    collection_name="pg_data",
    query_vector=query_vector,
    limit=3,
)

for r in results:
    print(f"Score: {r.score:.4f} | {r.payload['title']}: {r.payload['description']}")